In [1]:
from SD_IB_IRP_PPenv import steroid_IRP
import gurobipy as gu
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
env_config = {  'M': 6, 
                'K': 4, 
                'T': 4, 
                'F': 4, 
                
                'min_sprice': 1, 
                'max_sprice': 500, 
                'min_hprice': 1, 
                'max_hprice': 500,
                
                'S': 1, 
                'LA_horizon': 3, 
                'lambda1': 0.5,

                'back_o_cost': 100000
            }

back_orders = True
rd_seed = 0

# print('forecast', env.sample_paths[('d',0)])
# #print(env.d)
# print('real', env.d_t)

#print(env.Ages)

In [4]:
def rolling_horizon_det(state, _, env) :

    solucionTTP = {0:[  np.zeros(env.M+1, dtype=bool), 
                        np.zeros(env.M+1, dtype=int), 
                        np.zeros((env.M+1, env.K), dtype=bool), 
                        np.zeros((env.M+1, env.K), dtype=int), 
                        np.full(env.M+1, -1, dtype = int), 
                        np.zeros(env.M+1, dtype=int), 
                        np.zeros(env.K, dtype=int), 0, 0]}

    # State 
    I_0 = state.copy()
    sample_paths = _['sample_paths']

    # Look ahead window     
    Num_periods = env.sample_path_window_size
    T = range(Num_periods)

    # Iterables
    M = env.Suppliers;  K = env.Products

    # Initialization routing cost
    C_MIP = {(i,t):env.c[0,i]+env.c[i,0] for t in T for i in env.Suppliers} 

    m = gu.Model('Inventory')

    # Variables    
    # How much to buy from supplier i of product k at time t 
    z = {(i,k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="z_"+str((i,k,t))) for t in T for k in K for i in sample_paths[('M_k',0)][(k,t)]}

    # 1 if supplier i is selected at time t, 0 otherwise
    w = {(i,t):m.addVar(vtype=gu.GRB.BINARY, name="w_"+str((i,t))) for t in T for i in M}

    # Final inventory of product k of old o at time t 
    ii = {(k,t,o):m.addVar(vtype=gu.GRB.CONTINUOUS, name="i_"+str((k,t,o))) for k in K for t in T for o in range(env.O_k[k] + 1)}

    # Units sold of product k at time t of old age o
    y = {(k,t,o):m.addVar(vtype=gu.GRB.CONTINUOUS, name="y_"+str((k,t,o))) for k in K for t in T for o in range(env.O_k[k] + 1)}

    # Units in backorders of product k at time t
    bo = {(k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="bo_"+str((k,t))) for t in T for k in K}

    #Inventory constrains
    for k in K:
        for t in T:
            m.addConstr(ii[k,t,0] == gu.quicksum(z[i,k,t] for i in sample_paths[('M_k',0)][(k,t)]) - y[k,t,0])
            
    for k in K:
        for o in env.Ages[k]:
            m.addConstr(ii[k,0,o] == I_0[k,o] - y[k,0,o])
            
    for k in K:
        for t in T:
            for o in env.Ages[k]:
                if t > 0:
                    m.addConstr(ii[k,t,o] == ii[k,t-1,o-1] - y[k,t,o])

    for k in K: 
        for t in T:
            m.addConstr(gu.quicksum(y[k,t,o] for o in range(env.O_k[k] + 1)) + bo[k,t] == sample_paths[('d',0)][k,t])   


    #Purchase constrains
    for t in T:
        for k in K:
            for i in sample_paths[('M_k',0)][k,t]: 
                m.addConstr(z[i,k,t] <= env.sample_paths[('q',0)][i,k,t]*w[i,t])
                
    for t in T:
        for i in env.Suppliers:
            m.addConstr(gu.quicksum( z[i,k,t] for k in K if (i,k,t) in z) <= env.Q)
    
    compra = gu.quicksum(sample_paths['p',0][i,k,t]*z[i,k,t] for k in K for t in T for i in sample_paths[('M_k',0)][k,t] ) + \
        gu.quicksum(env.back_o_cost*bo[k,t] for k in K for t in T)
    
    #for the following H periods I will work with Expected Value
    ruta = gu.quicksum(C_MIP[i,t]*w[i,t] for i in M for t in T) 
        
    m.setObjective(compra+ruta)
            
    m.update()
    m.setParam('OutputFlag',0)
    m.optimize()

    # Purchase
    purchase = {(i,k): 0 for i in M for k in K}

    for k in K:
        for i in sample_paths[('M_k',0)][k,0]:
            purchase[i,k] = z[i,k,0].x
            if purchase[i,k]>0:
                solucionTTP[0][0][i] = True
                solucionTTP[0][1][i]+= purchase[i,k]
                solucionTTP[0][2][i][k]=True
                solucionTTP[0][3][i][k]=purchase[i,k]
                solucionTTP[0][6][k]+=purchase[i,k]

    # Demand compliance
    demand_compliance = {(k,o):y[k,0,o].x for k in K for o in range(env.O_k[k] + 1)}

    # Back-orders
    double_check = {(k,t): bo[k,0].x for k in K}
    
    #Updated inventory for next period t
    I_1 = {}
    for k in env.Products:
        for o in range(env.O_k[k] + 1):
            I_1[k,o] = ii[k,0,o].x

    Rutas_finales, solucionTTP, solucionTTP[0][8]  = Genera_ruta_at_t(solucionTTP, 0, max(env.c.values())*2, env.c, env.Q)

    return Rutas_finales, purchase, demand_compliance, double_check, I_1

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-19
Routes [[0, 4, 3, 2, 5, 0]] 
 
 
 Purchase {(1, 0): 0, (1, 1): 0.0, (1, 2): 0, (1, 3): 0, (2, 0): 0, (2, 1): 11.0, (2, 2): 4.0, (2, 3): 0, (3, 0): 0, (3, 1): 2.0, (3, 2): 0, (3, 3): 4.0, (4, 0): 0, (4, 1): 0, (4, 2): 10.0, (4, 3): 0, (5, 0): 0, (5, 1): 9.0, (5, 2): 10.0, (5, 3): 1.0} 
 
 
 Demand_compliance {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (1, 0): 20.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (2, 0): 17.0, (2, 1): 0.0, (3, 0): 4.5, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0} 
 
 
 Back orders {(0, 2): 0.0, (1, 2): 0.0, (2, 2): 0.0, (3, 2): 0.0}
Environment {(0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (1, 1): 2.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (2, 1): 7.0, (3, 1): 0.5, (3, 2): 0.0, (3, 3): 0.0}
Model {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (1, 0): 2.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (2, 0): 7.0,

In [ ]:
env = steroid_IRP(back_orders = back_orders, rd_seed = rd_seed, env_config = env_config)
state, _ = env.reset(return_state = True)         

Rutas_finales, purchase, demand_compliance, double_check, I_1 = rolling_horizon_det(state, _, env)
#print(Rutas_finales)
rutas = []
for key in Rutas_finales[0].keys():
    rutas.append(Rutas_finales[0][key][0])
#print(rutas)
print('Routes', rutas, '\n \n \n Purchase', purchase, '\n \n \n Demand_compliance', demand_compliance, '\n \n \n Back orders', double_check)

X = [rutas, purchase, demand_compliance]

state, reward, done, _ = env.step(X)
print('Environment', state)
print('Model', I_1)

In [ ]:
def Deterministic_Rolling_Horizon(Vertex, Products, Periods, V, M, K, T, Q, h, O_k, Mk, d, q, p, q_disp, I_0, c, max_cij, dist_demand_parm, d_EV, q_EV, p_EV, H, path_eval):
    
    final_policy = {} 
    FO_policy = 0
    I_0 = {(k,o):0 for k in K for o in range(O_k[k])} #Initial inventory level                                     
    
    #Encoding purchase and routing solution for TPP at all times t
    solucionTTP = {t:[np.zeros(len(V), dtype=bool), np.zeros(len(V), dtype=int), np.zeros((len(V), len(K)), dtype=bool), np.zeros((len(V), len(K)), dtype=int), np.full(len(V) , -1, dtype = int), np.zeros(len(V), dtype=int), np.zeros(len(K), dtype=int), 0, 0]   for t in T}
    #[t][0] bool vector that said if supplier i is visited at time t
    #[t][1] int vector that said how much is total purchased in supplier i at time t
    #[t][2] bool matrix that said if product k is purchased in supplier i at time t
    #[t][3] int matrix that said how much is purchased of product k in supplier i at time t
    #[t][4] int vector that said in which route is supplier i at time t
    #[t][5] int vetor that said in which route position is supplier i at time t
    #[t][6] int vector that said how much is total purchased of product k at time t
    #[t][7] Purchase cost at time t
    #[t][8] Routing cost at time t
    
    #Encoding backorders of product k at time t
    compra_extra = {t:np.zeros(len(K), dtype = int) for t in T}
    
    #Encoding initial and final inventory levels for each old
    inventario = {t:[[[0 for o in range(O_k[k]+1)] for k in K], [[0 for o in range(O_k[k]+1)] for k in K]]  for t in range(len(T)+1)}                 
    #[t][0] initial inventory for each old
    #[t][1] Final inventory for each old
    
    
    for t_RH in T: 
        
        #The window to be evaluated is created
        if t_RH + H < len(T)-1:
            
            t_max = t_RH+H
        else:
            t_max=len(T)-1
        
        TT = range(t_RH,t_max+1)
        
        C_MIP = {(i,t):c[0,i]+c[i,0] for t in TT for i in M} #Initialization routing cost
        
        #Build model with specific window
        
        m = gu.Model('Inventory')
        
        #Sets
        #V: Vertex
        #M: markets
        #K: Products
        #TT: Periods into windows t_RH + H 
        
    
        #Variables    
        #How much to buy from supplier i of product k at time t within window t_RH + H
        z = {(i,k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="z_"+str((i,k,t))) for t in TT for k in K for i in Mk[k,t]}
        
        #1 if supplier i is selected at time t, 0 otherwise within window t_RH + H
        w = {(i,t):m.addVar(vtype=gu.GRB.BINARY, name="w_"+str((i,t))) for t in TT for i in M}
        
        #Final inventory of product k of old o at time t within window t_RH + H
        ii = {(k,t,o):m.addVar(vtype=gu.GRB.CONTINUOUS, name="i_"+str((k,t,o))) for k in K for t in TT for o in range(O_k[k])}
        
        #Units sold of product k at time t of old age o
        y = {(k,t,o):m.addVar(vtype=gu.GRB.CONTINUOUS, name="y_"+str((k,t,o))) for k in K for t in TT for o in range(O_k[k])}
        
        #Units in backorders of product k at time t
        bo = {(k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="bo_"+str((k,t))) for t in TT for k in K}
        
        #Inventory constrains
        for k in K:
            for t in TT:
                m.addConstr(ii[k,t,0] == gu.quicksum(z[i,k,t] for i in Mk[k,t]) -y[k,t,0])
                
        for k in K:
            for o in range(1, O_k[k]):
                m.addConstr(ii[k,t_RH,o] == I_0[k,o-1]-y[k,t_RH,o])
                
        for k in K:
            for t in TT:
                for o in range(O_k[k]):
                    if t>t_RH and o >0:
                        m.addConstr(ii[k,t,o] == ii[k,t-1,o-1]-y[k,t,o])     
                        
        for k in K:
            
            #Standard deviation according to demand distribution
            n = dist_demand_parm[k][1] - dist_demand_parm[k][0] + 1
            std_value = np.sqrt( ((n**2)-1)/12) 
            
            for t in TT:
                if t == t_RH:
                    #Today I know the demand
                    m.addConstr(gu.quicksum(y[k,t,o] for o in range(O_k[k])) + bo[k,t] == d[k,t,path_eval]) 
                else:
                    #for the following H periods I will work with Expected Value
                    #m.addConstr(gu.quicksum(y[k,t,o] for o in range(O_k[k])) + bo[k,t] == int(d_EV[k,t]+std_value) ) 
                    m.addConstr(gu.quicksum(y[k,t,o] for o in range(O_k[k])) + bo[k,t] == int(d_EV[k,t]) ) 
                                    #I'm working with EV + 1 standar desviation because produces better resutls. 
        
        #Purchase constrains
        for t in TT:
            for k in K:
                for i in Mk[k,t]:
                    if t == t_RH:
                        #Today I know the available capacity
                        m.addConstr(z[i,k,t] <= q[i,k,t,path_eval]*w[i,t])
                    else:
                        #for the following H periods I will work with Expected Value
                        m.addConstr(z[i,k,t] <= q_EV[i,k,t]*w[i,t])
                    
        for t in TT:
            for i in M:
                m.addConstr(gu.quicksum( z[i,k,t] for k in K if (i,k,t) in z) <= Q )
            
        #gu.quicksum(h[k,t]*ii[k,t,o] for o in range(1, O_k[k]+1))
        
        #I know the price today
        compra_dia_t = gu.quicksum(p[i,k,t_RH,path_eval]*z[i,k,t_RH] for k in K for i in Mk[k,t_RH]) + gu.quicksum(1000*bo[k,t_RH] for k in K)
        
        #for the following H periods I will work with Expected Value
        compa_other_days = gu.quicksum(p_EV[i,k,t]*z[i,k,t] for k in K for t in range(t_RH+1,t_max+1) for i in Mk[k,t] ) + gu.quicksum(1000*bo[k,t] for k in K for t in range(t_RH+1,t_max+1))
        ruta = gu.quicksum(C_MIP[i,t]*w[i,t] for i in M for t in TT) 
            
        m.setObjective(compra_dia_t+compa_other_days+ruta)
                
        m.update()
        m.setParam('OutputFlag',0)
        m.optimize()
           
        if m.Status==2 or m.Status==9:
            
            of_full_aproximation = m.ObjVal
            var_compra = {(i,k,t):z[i,k,t].x for t in TT for k in K for i in Mk[k,t]}
            
            total_compra_MIP = sum(p[i,k,t_RH,path_eval]*z[i,k,t_RH].x for k in K for i in Mk[k,t_RH]) + sum(p_EV[i,k,t]*z[i,k,t].x for k in K for t in range(t_RH+1,t_max+1) for i in Mk[k,t] )
            total_ruta_MIP = sum(C_MIP[i,t]*w[i,t].x for i in M for t in TT)
            
            for k in K:
                compra_extra[t_RH][k] = bo[k,t_RH].x
                for o in range(O_k[k]):
                    inventario[t_RH][0][k][o] = I_0[k,o]
                    inventario[t_RH][1][k][o] = ii[k,t_RH,o].x
                    
            for k in K:
                for i in Mk[k,t_RH]:
                    if var_compra[i,k,t_RH] > 0:
                        solucionTTP[t_RH][0][i] = True
                        solucionTTP[t_RH][1][i]+= var_compra[i,k,t_RH]
                        solucionTTP[t_RH][2][i][k]=True
                        solucionTTP[t_RH][3][i][k]=var_compra[i,k,t_RH]
                        solucionTTP[t_RH][6][k]+=var_compra[i,k,t_RH]
                        
            Rutas_finales, solucionTTP, solucionTTP[t_RH][8]  = Genera_ruta_at_t(solucionTTP, t_RH, max_cij, c, Q)
                    
            solucionTTP[t_RH].append(Rutas_finales.copy())
            
            costo_compra_dia_t = sum(p[i,k,t_RH,path_eval]*z[i,k,t_RH].x for k in K for i in Mk[k,t_RH])
            solucionTTP[t_RH][7] = costo_compra_dia_t
            costo_ruta_dia_t = solucionTTP[t_RH][8]
            
            #Build policy at time t
            final_policy[t_RH]=(solucionTTP[t_RH].copy(), inventario[t_RH].copy(), compra_extra[t_RH], costo_compra_dia_t, costo_ruta_dia_t, costo_compra_dia_t+costo_ruta_dia_t)
            FO_policy += costo_compra_dia_t+costo_ruta_dia_t
            
            
            #Updated inventory for next period t
            for k in K:
                for o in range(O_k[k]):
                    I_0[k,o] = ii[k,t_RH,o].x
                          
        else:
            print('Problem infeasible')
            
    return final_policy, FO_policy

In [ ]:
def Model_Find_Q_with_demand_stochastic(T, K, M, Mk, d, q):
    
    m = gu.Model('Model_Find_Q_with_demand')

    z = {(i,k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="z_"+str((i,k,t))) for t in T for k in K for i in Mk[k,t]}
    w = {(i,t):m.addVar(vtype=gu.GRB.BINARY, name="w_"+str((i,t))) for t in T for i in M}
    y = {(k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="y_"+str((k,t))) for k in K for t in T}
    Q = m.addVar(vtype=gu.GRB.CONTINUOUS)
    
    for k in K:
        for t in T:
            m.addConstr(gu.quicksum(z[i,k,t] for i in Mk[k,t])+y[k,t] == d[k,t])        
            
    for t in T:
        for k in K:
            for i in Mk[k,t]:
                m.addConstr(z[i,k,t] <= q[i,k,t]*w[i,t])
                
    for t in T:
        for i in M:
            m.addConstr(gu.quicksum( z[i,k,t] for k in K if (i,k,t) in z) <= Q  )
        
    m.setObjective(gu.quicksum(10000*y[k,t] for k in K for t in T))
            
    m.update()
    m.setParam('OutputFlag',0)
    m.optimize()
    
    return Q.x

In [ ]:
def Stochastic_Instance_Generator(Vertex, Products, Periods, paths_evaluation, veh_factor, seedd, sample_paths):

    np.random.seed(seedd)
                    
    nombre = "MVTPP_IM_"+str(Vertex)+"_"+str(Products)+"_"+str(Periods)+"_"+str(seedd)+".txt"
    print(nombre)
    
    M = range(1, Vertex) #Set of farmers
    V = range(Vertex) #Set of vertexs M U {0} Depot
    K = range(Products)  #Set of products
    T = range(Periods) #Set of periods

    size_grid = 1000 #Grid Size 

    #Farmers and depot coordinates
    coor = {i:(np.random.randint(0, size_grid), np.random.randint(0, size_grid)) for i in range(Vertex)}
    
    #the farmers offering the products are the same for all periods 
    Mk = {} 
    t = 0
    Markets = []
    for k in K:
        if (k,t) not in Mk.keys():
            Mk[k,t]=[]
        MarkesLen = np.random.randint(np.ceil(Vertex*0.5), Vertex) #at least 50% of the suppliers are offering products
        while len(Mk[k,t]) < MarkesLen:
            seleccion = np.random.randint(1, Vertex)
            if seleccion not in Mk[k,t]:
                Mk[k,t].append(seleccion)
                if seleccion not in Markets:
                    Markets.append(seleccion)
                    
    #If a supplier does not offer any product, a product is randomly assigned to that supplier.
    for i in M:
        if i not in Markets:
            selecion = np.random.randint(Products)
            Mk[selecion,t].append(i)

    for k in K:
        Mk[k,t].sort()
    
    #The suppliers and products offered for the other periods are replicated.
    for t in range(1, len(T)):
        for k in K:
            Mk[k,t] = Mk[k,0].copy()
    
    p = {}
    q = {}              

    p_EV = {}
    q_EV = {}

    p_sample_paths = {}
    q_sample_paths = {}
    
    #Upper limit for each supplier and product
    rang_max_p = {(i,k):np.random.randint(500, 801) for k in K for i in Mk[k,0]}
    rang_max_q = {(i,k):np.random.randint(15, 61) for k in K for i in Mk[k,0]}
    
    for t in T:
        for k in K:
            for i in Mk[k,t]:
                
                #Expected value of the price and availability according to the stipulated limits.
                p_EV[i,k,t] = int(round((rang_max_p[i,k]-1)/2,0))
                q_EV[i,k,t] = int(round((rang_max_q[i,k]-1)/2,0))
                
                #Real values of price and availability for each path evaluation (This is for policy evaluation)
                for path_eval in range(paths_evaluation):
                    p[i,k,t,path_eval] = np.random.randint(1, rang_max_p[i,k])
                    q[i,k,t,path_eval] = np.random.randint(1, rang_max_q[i,k])
                
                    #Realization for each sample path according to the values of the distribution
                    for path in range(sample_paths):
                        p_sample_paths[i,k,t,path_eval,path] = np.random.randint(1, rang_max_p[i,k])
                        q_sample_paths[i,k,t,path_eval,path] = np.random.randint(1, rang_max_q[i,k])
    hh = {}
    for k in K:
        lista_ayuda = []
        for i in Mk[k,0]:
            lista_ayuda.append(p_EV[i,k,0])

        promedio = np.average(lista_ayuda)

        for t in T:
            hh[k,t] = np.ceil(promedio)
    
    #Maximum value for the perishability of each product
    O_k = {k:np.random.randint(1, len(T)+1) for k in K}
    
    #Demand distribution is built
    dist_demand_parm = {}
    t = 0
    for k in K:
        maximo = 0
        suma = 0
        for i in Mk[k,t]:
            for path_eval in range(paths_evaluation):
                suma+=q[i,k,t,path_eval]
                if q[i,k,t,path_eval] > maximo:
                    maximo = q[i,k,t,path_eval]
                    
        dist_demand_parm[k] = [maximo, int(suma/paths_evaluation)] 
    
    d = {}
    d_EV = {}
    d_sample_paths = {}

    for k in K:
        for t in T:
            
            d_EV[k,t] = int(round((dist_demand_parm[k][1]-dist_demand_parm[k][0])/2,0)) 
            
            for path_eval in range(paths_evaluation):
                d[k,t,path_eval] = np.random.randint(dist_demand_parm[k][0], dist_demand_parm[k][1])

                for path in range(sample_paths):
                    d_sample_paths[k,t,path_eval,path] = np.random.randint(dist_demand_parm[k][0], dist_demand_parm[k][1])


    I_0 = {(k,o):0 for k in K for o in range(O_k[k])} #Initial inventory level                                     

    Q = Model_Find_Q_with_demand_stochastic(T, K, M, Mk, d_EV, q_EV)
    
    Q = Q*veh_factor

    
    #For the moment I increase capacity to be feasible 
    for t in T:
        for k in K:
            for i in Mk[k,t]:
                q_EV[i,k,t] += 2*q_EV[i,k,t]
                
                for path_eval in range(paths_evaluation):
                    q[i,k,t,path_eval] += 2*q[i,k,t,path_eval]
    
    q_disp = q.copy()
    #Travel cost between (i,j). It is the same for all time t
    c = {(i,j):round(np.sqrt(((coor[i][0]-coor[j][0])**2)+((coor[i][1]-coor[j][1])**2)),0) for t in T for i in range(Vertex) for j in range(Vertex) if i!=j }
    
    max_cij = max(c.values())*2
    
    return Vertex, Products, Periods, M, V, K, T, coor, Mk, Q, O_k, hh, d, d_EV, d_sample_paths, q, q_EV, q_sample_paths, p,p_EV, p_sample_paths, q_disp, I_0, c, max_cij, paths_evaluation, dist_demand_parm  

In [3]:
class Graph: # Class to represent a graph
 
    def __init__(self, vertices):
        self.V = vertices # No. of vertices
        self.graph = []
 
    # function to add an edge to graph
    def addEdge(self, u, v, w):
        self.graph.append([u, v, w])
         
    # utility function used to print the solution
    def printArr(self, dist, etiqueta):
        print("Vertex Distance from Source")
        for i in range(self.V):
            print("{0}\t\t{1}\t\t{2}".format(i, dist[i], etiqueta[i]))
     
    # The main function that finds shortest distances from src to
    # all other vertices using Bellman-Ford algorithm. The function
    # also detects negative weight cycle
    def BellmanFord(self, src):
 
        # Step 1: Initialize distances from src to all other vertices
        # as INFINITE
        dist = [float("Inf")] * self.V
        etiqueta = [0]*self.V
        dist[src] = 0
        
        # Step 2: Relax all edges |V| - 1 times. A simple shortest
        # path from src to any other vertex can have at-most |V| - 1
        # edges
        for _ in range(self.V - 1):
            # Update dist value and parent index of the adjacent vertices of
            # the picked vertex. Consider only those vertices which are still in
            # queue
            for u, v, w in self.graph:
                if dist[u] != float("Inf") and dist[u] + w < dist[v]:
                        dist[v] = dist[u] + w
                        etiqueta[v] = u
                        
        
        # print all distance
        #self.printArr(dist, etiqueta)
        
        Cabeza = -1
        Cola = self.V-1
        FO = dist[Cola]
        arcos = []
        while Cabeza !=0 :
            Cabeza = etiqueta[Cola]
            arcos.append((Cabeza,Cola))
            Cola = Cabeza
        
        #print(FO)
        #print(arcos)
        return FO, arcos
    
    
def Crea_grafo_aumentado_at_t(t, solucionTTP, max_cij, c):
    Info_Route = {}
    nodes = [i for i, x in enumerate(solucionTTP[t][0]) if x] #See which suppliers are in the solution
    nodes.insert(0,0)
    
    if len(nodes)>1: 
        Info_Route[t] = [nodes, len(nodes)]
        
        matrix_cij = np.full((Info_Route[t][1],Info_Route[t][1]), max_cij) 
        
        #Build distance matrix just for supplier selected
        for i in range(len(Info_Route[t][0])):
            for j in range(len(Info_Route[t][0])):
                if i != j :
                    matrix_cij[i,j] = c[Info_Route[t][0][i],Info_Route[t][0][j]]
                    
        #Run nearest neighbor algorithm
        Ruta = []            
        Cabeza = 0 
        Ruta.append(Cabeza)
        
        while len(Ruta) < Info_Route[t][1]:
            minimo = max_cij
            for i in range(len(nodes)):
                if matrix_cij[Cabeza][i] < minimo:
                    minimo = matrix_cij[Cabeza][i]
                    Cola = i
                
                matrix_cij[i][Cabeza]=max_cij
                
            Ruta.append(Info_Route[t][0][Cola])
            Cabeza = Cola
            
        Info_Route[t].append(Ruta) #General route
    
    return Info_Route, solucionTTP
    
#Build Augmented graph following general tour
def Genera_Rutas_CVRP_at_t(Info_Route, solucionTTP, t, c, Q):
    Rutas_finales = {}
    FO_Rutas = {}
    Rutas_finales[t] = {}
    info = {}
    
    if len(Info_Route) > 0: 
        g = Graph(Info_Route[t][1]) #Define the number of suppliers
        
        #Build the arcs just if they respect vehicle capacity following the general route found
        for i in range(Info_Route[t][1]-1):
            for j in range(i+1, Info_Route[t][1]): 
                info_ruta = {}   
                
                if j - i == 1:
                    camino = [0, Info_Route[t][2][j] ,0]
                    costo = sum(c[camino[k], camino[k+1]] for k in range(len(camino)-1))
                    cap = solucionTTP[t][1][camino[1]]
                    info_ruta[camino[1]] = 1
            
                    if cap <= Q:
                        info[(i,j)] = [camino, cap, costo, info_ruta]
                        g.addEdge(i,j,costo)
                        
                elif j - i >1:
                    if i ==0:
                        camino=Info_Route[t][2][i:j+1]
                        camino.append(0)
                    else:
                        camino=Info_Route[t][2][i+1:j+1]
                        camino.append(0)
                        camino.insert(0,0)
                    
                    for k in range(1,len(camino)-1):
                        info_ruta[camino[k]] = k
                
                    costo = sum(c[camino[k], camino[k+1]] for k in range(len(camino)-1))
                    cap = sum(solucionTTP[t][1][camino[k]]  for k in range(1, len(camino)))
                    
                    if cap <=Q:
                        info[(i,j)] = [camino, cap, costo, info_ruta]
                        g.addEdge(i,j,costo)
        
        FO_Routing, Arcos_agregados = g.BellmanFord(0) #Run BellamnFord Algorithm 
        
        #Translate arc in routing solution
        for i in range(len(Arcos_agregados)):
            Rutas_finales[t][i] = info[Arcos_agregados[i]]
            for k in info[Arcos_agregados[i]][3]:
                solucionTTP[t][4][k]=i
                solucionTTP[t][5][k] = info[Arcos_agregados[i]][3][k]
                
    else:
        FO_Routing = 0
            
    return FO_Routing, Rutas_finales

def Genera_ruta_at_t(solucionTTP, t, max_cij, c, Q):
    
    Info_Route, solucionTTP = Crea_grafo_aumentado_at_t(t, solucionTTP, max_cij, c)
    FO_Routing, Rutas_finales = Genera_Rutas_CVRP_at_t(Info_Route, solucionTTP, t, c, Q) 
    
    return Rutas_finales, solucionTTP, FO_Routing

\begin{equation}
\min \sum_{t \in T}\left( \sum_{i\in M }\hat{c_{it}}w_{it} + \sum_{k\in K }\sum_{i\in M_{k}}p_{ikt}z_{ikt}+ \sum_{k\in K }\sum_{o\in O_{k}}h_{kt}I_{kto}   \right)
\end{equation}


Subject to

\begin{align}
& I_{kt0} =  \sum_{i\in M_{k}}z_{ikt}-y_{kt0} & &
&,\forall k \in K, \forall t \in T
\end{align}

\begin{align}
& I_{k1o} =  I_{k0o} - y_{k1o} & &
&,\forall k \in K, \forall o \in O_{k} | o >0
\end{align}

\begin{align}
& I_{kto} =  I_{kt-1o-1} - y_{kto} & &
&,\forall k \in K, \forall t \in T | t >1, \forall o \in O_{k}| o > 0
\end{align}

\begin{align}
& \sum_{o\in O_{k}}y_{kto} =  d_{kt} & & &,\forall k \in K, \forall t \in T
\end{align}

\begin{align}
& z_{ikt} \leq q_{ikt}w_{it} & & &,\forall k \in K, \forall t \in T
\end{align}

\begin{align}
& \sum_{k \in K}\sum_{i \in M_{kt}} z_{ikt} \leq Q & & &,\forall t \in T
\end{align}

\begin{align}
& I_{kto} \geq 0   & & &
,\forall k \in K, \forall t \in T, \forall o \in O_{k}
\end{align}

\begin{align}
& y_{kto} \geq 0   & & &
,\forall k \in K, \forall t \in T, \forall o \in O_{k}
\end{align}

\begin{align}
& w_{it} \in \{0,1\}   & & &
,\forall i \in M,  \forall t \in T
\end{align}

\begin{align}
& z_{ikt} \geq 0   & & &
,\forall i \in M_{k,t},\forall k \in K, \forall t \in T \end{align}

In [ ]:
def Deterministic_Rolling_Horizon(Vertex, Products, Periods, V, M, K, T, Q, h, O_k, Mk, d, q, p, q_disp, I_0, c, max_cij, dist_demand_parm, d_EV, q_EV, p_EV, H, path_eval):
    
    final_policy = {} 
    FO_policy = 0
    I_0 = {(k,o):0 for k in K for o in range(O_k[k])} #Initial inventory level                                     
    
    #Encoding purchase and routing solution for TPP at all times t
    solucionTTP = {t:[np.zeros(len(V), dtype=bool), np.zeros(len(V), dtype=int), np.zeros((len(V), len(K)), dtype=bool), np.zeros((len(V), len(K)), dtype=int), np.full(len(V) , -1, dtype = int), np.zeros(len(V), dtype=int), np.zeros(len(K), dtype=int), 0, 0]   for t in T}
    #[t][0] bool vector that said if supplier i is visited at time t
    #[t][1] int vector that said how much is total purchased in supplier i at time t
    #[t][2] bool matrix that said if product k is purchased in supplier i at time t
    #[t][3] int matrix that said how much is purchased of product k in supplier i at time t
    #[t][4] int vector that said in which route is supplier i at time t
    #[t][5] int vetor that said in which route position is supplier i at time t
    #[t][6] int vector that said how much is total purchased of product k at time t
    #[t][7] Purchase cost at time t
    #[t][8] Routing cost at time t
    
    #Encoding backorders of product k at time t
    compra_extra = {t:np.zeros(len(K), dtype = int) for t in T}
    
    #Encoding initial and final inventory levels for each old
    inventario = {t:[[[0 for o in range(O_k[k]+1)] for k in K], [[0 for o in range(O_k[k]+1)] for k in K]]  for t in range(len(T)+1)}                 
    #[t][0] initial inventory for each old
    #[t][1] Final inventory for each old
    
    
    for t_RH in T: 
        
        #The window to be evaluated is created
        if t_RH + H < len(T)-1:
            
            t_max = t_RH+H
        else:
            t_max=len(T)-1
        
        TT = range(t_RH,t_max+1)
        
        C_MIP = {(i,t):c[0,i]+c[i,0] for t in TT for i in M} #Initialization routing cost
        
        #Build model with specific window
        
        m = gu.Model('Inventory')
        
        #Sets
        #V: Vertex
        #M: markets
        #K: Products
        #TT: Periods into windows t_RH + H 
        
    
        #Variables    
        #How much to buy from supplier i of product k at time t within window t_RH + H
        z = {(i,k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="z_"+str((i,k,t))) for t in TT for k in K for i in Mk[k,t]}
        
        #1 if supplier i is selected at time t, 0 otherwise within window t_RH + H
        w = {(i,t):m.addVar(vtype=gu.GRB.BINARY, name="w_"+str((i,t))) for t in TT for i in M}
        
        #Final inventory of product k of old o at time t within window t_RH + H
        ii = {(k,t,o):m.addVar(vtype=gu.GRB.CONTINUOUS, name="i_"+str((k,t,o))) for k in K for t in TT for o in range(O_k[k])}
        
        #Units sold of product k at time t of old age o
        y = {(k,t,o):m.addVar(vtype=gu.GRB.CONTINUOUS, name="y_"+str((k,t,o))) for k in K for t in TT for o in range(O_k[k])}
        
        #Units in backorders of product k at time t
        bo = {(k,t):m.addVar(vtype=gu.GRB.CONTINUOUS, name="bo_"+str((k,t))) for t in TT for k in K}
        
        #Inventory constrains
        for k in K:
            for t in TT:
                m.addConstr(ii[k,t,0] == gu.quicksum(z[i,k,t] for i in Mk[k,t]) -y[k,t,0])
                
        for k in K:
            for o in range(1, O_k[k]):
                m.addConstr(ii[k,t_RH,o] == I_0[k,o-1]-y[k,t_RH,o])
                
        for k in K:
            for t in TT:
                for o in range(O_k[k]):
                    if t>t_RH and o >0:
                        m.addConstr(ii[k,t,o] == ii[k,t-1,o-1]-y[k,t,o])     
                        
        for k in K:
            
            #Standard deviation according to demand distribution
            n = dist_demand_parm[k][1] - dist_demand_parm[k][0] + 1
            std_value = np.sqrt( ((n**2)-1)/12) 
            
            for t in TT:
                if t == t_RH:
                    #Today I know the demand
                    m.addConstr(gu.quicksum(y[k,t,o] for o in range(O_k[k])) + bo[k,t] == d[k,t,path_eval]) 
                else:
                    #for the following H periods I will work with Expected Value
                    #m.addConstr(gu.quicksum(y[k,t,o] for o in range(O_k[k])) + bo[k,t] == int(d_EV[k,t]+std_value) ) 
                    m.addConstr(gu.quicksum(y[k,t,o] for o in range(O_k[k])) + bo[k,t] == int(d_EV[k,t]) ) 
                                    #I'm working with EV + 1 standar desviation because produces better resutls. 
        
        #Purchase constrains
        for t in TT:
            for k in K:
                for i in Mk[k,t]:
                    if t == t_RH:
                        #Today I know the available capacity
                        m.addConstr(z[i,k,t] <= q[i,k,t,path_eval]*w[i,t])
                    else:
                        #for the following H periods I will work with Expected Value
                        m.addConstr(z[i,k,t] <= q_EV[i,k,t]*w[i,t])
                    
        for t in TT:
            for i in M:
                m.addConstr(gu.quicksum( z[i,k,t] for k in K if (i,k,t) in z) <= Q )
            
        #gu.quicksum(h[k,t]*ii[k,t,o] for o in range(1, O_k[k]+1))
        
        #I know the price today
        compra_dia_t = gu.quicksum(p[i,k,t_RH,path_eval]*z[i,k,t_RH] for k in K for i in Mk[k,t_RH]) + gu.quicksum(1000*bo[k,t_RH] for k in K)
        
        #for the following H periods I will work with Expected Value
        compa_other_days = gu.quicksum(p_EV[i,k,t]*z[i,k,t] for k in K for t in range(t_RH+1,t_max+1) for i in Mk[k,t] ) + gu.quicksum(1000*bo[k,t] for k in K for t in range(t_RH+1,t_max+1))
        ruta = gu.quicksum(C_MIP[i,t]*w[i,t] for i in M for t in TT) 
            
        m.setObjective(compra_dia_t+compa_other_days+ruta)
                
        m.update()
        m.setParam('OutputFlag',0)
        m.optimize()
           
        if m.Status==2 or m.Status==9:
            
            of_full_aproximation = m.ObjVal
            var_compra = {(i,k,t):z[i,k,t].x for t in TT for k in K for i in Mk[k,t]}
            
            total_compra_MIP = sum(p[i,k,t_RH,path_eval]*z[i,k,t_RH].x for k in K for i in Mk[k,t_RH]) + sum(p_EV[i,k,t]*z[i,k,t].x for k in K for t in range(t_RH+1,t_max+1) for i in Mk[k,t] )
            total_ruta_MIP = sum(C_MIP[i,t]*w[i,t].x for i in M for t in TT)
            
            for k in K:
                compra_extra[t_RH][k] = bo[k,t_RH].x
                for o in range(O_k[k]):
                    inventario[t_RH][0][k][o] = I_0[k,o]
                    inventario[t_RH][1][k][o] = ii[k,t_RH,o].x
                    
            for k in K:
                for i in Mk[k,t_RH]:
                    if var_compra[i,k,t_RH] > 0:
                        solucionTTP[t_RH][0][i] = True
                        solucionTTP[t_RH][1][i]+= var_compra[i,k,t_RH]
                        solucionTTP[t_RH][2][i][k]=True
                        solucionTTP[t_RH][3][i][k]=var_compra[i,k,t_RH]
                        solucionTTP[t_RH][6][k]+=var_compra[i,k,t_RH]
                        
            Rutas_finales, solucionTTP, solucionTTP[t_RH][8]  = Genera_ruta_at_t(solucionTTP, t_RH, max_cij, c, Q)
                    
            solucionTTP[t_RH].append(Rutas_finales.copy())
            
            costo_compra_dia_t = sum(p[i,k,t_RH,path_eval]*z[i,k,t_RH].x for k in K for i in Mk[k,t_RH])
            solucionTTP[t_RH][7] = costo_compra_dia_t
            costo_ruta_dia_t = solucionTTP[t_RH][8]
            
            #Build policy at time t
            final_policy[t_RH]=(solucionTTP[t_RH].copy(), inventario[t_RH].copy(), compra_extra[t_RH], costo_compra_dia_t, costo_ruta_dia_t, costo_compra_dia_t+costo_ruta_dia_t)
            FO_policy += costo_compra_dia_t+costo_ruta_dia_t
            
            
            #Updated inventory for next period t
            for k in K:
                for o in range(O_k[k]):
                    I_0[k,o] = ii[k,t_RH,o].x
                          
        else:
            print('Problem infeasible')
            
    return final_policy, FO_policy

In [ ]:
def Stochasctic_Rolling_Horizon(Vertex, Products, Periods, Q, coor, h, d, O_k, Mk, q, p, M, V, K, T, q_disp, I_0, c, max_cij, dist_demand_parm, q_sample_paths, p_sample_paths, d_sample_paths, H, path_eval, sample_paths):
    
    S = range(sample_paths)
    I_0 = {(k,o):0 for k in K for o in range(O_k[k])} #Initial inventory level                                     
    
    final_policy = {}    
    FO_policy = 0
    ps = {s:1/len(S) for s in S}
    
    solucionTTP = {t:[np.zeros(len(V), dtype=bool), np.zeros(len(V), dtype=int), np.zeros((len(V), len(K)), dtype=bool), np.zeros((len(V), len(K)), dtype=int), np.full(len(V) , -1, dtype = int), np.zeros(len(V), dtype=int), np.zeros(len(K), dtype=int), 0, 0]   for t in T}
    compra_extra = {t:np.zeros(len(K), dtype = int) for t in T}
    inventario = {t:[[[0 for o in range(O_k[k]+1)] for k in K], [[0 for o in range(O_k[k]+1)] for k in K]]  for t in range(len(T)+1)}
    
    for t_RH in T:
        
        if t_RH + H < len(T)-1:
            
            t_max = t_RH+H
        else:
            t_max=len(T)-1
        
        TT = range(t_RH,t_max+1)
        
        C_MIP = {(i,t):c[0,i]+c[i,0] for t in TT for i in M }
        
        m = gu.Model('Inventory')
        
        #Inventory variables
        z = {(i,k,t,s):m.addVar(vtype=gu.GRB.CONTINUOUS, name="z_"+str((i,k,t,s))) for t in TT for k in K for i in Mk[k,t] for s in S}
        w = {(i,t,s):m.addVar(vtype=gu.GRB.BINARY, name="w_"+str((i,t,s))) for t in TT for i in M for s in S}
        ii = {(k,t,o,s):m.addVar(vtype=gu.GRB.CONTINUOUS, name="i_"+str((k,t,o,s))) for k in K for t in TT for o in range(O_k[k]) for s in S}
        y = {(k,t,o,s):m.addVar(vtype=gu.GRB.CONTINUOUS, name="y_"+str((k,t,o,s))) for k in K for t in TT for o in range(O_k[k]) for s in S}
        bo = {(k,t,s):m.addVar(vtype=gu.GRB.CONTINUOUS, name="bo_"+str((k,t,s))) for t in TT for k in K for s in S}
    
        for s in S:
        
            #Inventory constrains
            for k in K:
                for t in TT:
                    m.addConstr(ii[k,t,0,s] == gu.quicksum(z[i,k,t,s] for i in Mk[k,t]) -y[k,t,0,s])
                    
            for k in K:
                for o in range(1, O_k[k]):
                    m.addConstr(ii[k,t_RH,o,s] == I_0[k,o-1]-y[k,t_RH,o,s])
            
            for k in K:
                for t in TT:
                    for o in range(O_k[k]):
                        if t>t_RH and o >0:
                            m.addConstr(ii[k,t,o,s] == ii[k,t-1,o-1,s]-y[k,t,o,s])                
            
            for k in K:
                for t in TT:
                    if t == t_RH:
                        m.addConstr(gu.quicksum(y[k,t,o,s] for o in range(O_k[k])) + bo[k,t,s] == d[k,t,path_eval])
                    else:
                        m.addConstr(gu.quicksum(y[k,t,o,s] for o in range(O_k[k])) + bo[k,t,s] == d_sample_paths[k,t,path_eval,s])
                    
            for t in TT:
                for k in K:
                    for i in Mk[k,t]:
                        if t == t_RH:
                            m.addConstr(z[i,k,t,s] <= q[i,k,t,path_eval]*w[i,t,s])
                        else:
                            m.addConstr(z[i,k,t,s] <= q_sample_paths[i,k,t,path_eval,s]*w[i,t,s])
                            
            for t in TT:
                for i in M:
                    m.addConstr(gu.quicksum( z[i,k,t,s] for k in K if (i,k,t,s) in z   ) <= Q )
            
            for k in K:
                
                m.addConstr(bo[k,t_RH,s] ==  gu.quicksum(bo[k,t_RH,ss]/len(S) for ss in S))
                
                for i in Mk[k,t_RH]:
                    m.addConstr(z[i,k,t_RH,s] ==  gu.quicksum(z[i,k,t_RH,ss]/len(S) for ss in S))
                    
                for o in range(O_k[k]):
                    m.addConstr(ii[k,t_RH,o,s] ==  gu.quicksum(ii[k,t_RH,o,ss]/len(S) for ss in S))
                    m.addConstr(y[k,t_RH,o,s] ==  gu.quicksum(y[k,t_RH,o,ss]/len(S) for ss in S))
                     
            for i in M:
                m.addConstr(w[i,t_RH,s] ==  gu.quicksum(w[i,t_RH,ss]/len(S) for ss in S))
            
            
        #gu.quicksum(h[k,t]*ii[k,t,o] for o in range(1, O_k[k]+1))
        
        compra_dia_t = gu.quicksum(ps[s]*gu.quicksum(p[i,k,t_RH,path_eval]*z[i,k,t_RH,s] for k in K for i in Mk[k,t_RH]) + gu.quicksum(1000*bo[k,t_RH,s] for k in K) for s in S)
        ruta_dia_t = gu.quicksum(ps[s]*gu.quicksum(C_MIP[i,t_RH]*w[i,t_RH,s] for i in M) for s in S)
        
        compa_other_days = gu.quicksum(ps[s]*gu.quicksum(p_sample_paths[i,k,t,path_eval,s]*z[i,k,t,s] for k in K for t in range(t_RH+1,t_max+1) for i in Mk[k,t] ) + gu.quicksum(1000*bo[k,t,s] for k in K for t in range(t_RH+1,t_max+1)) for s in S)
        ruta_other_days = gu.quicksum(ps[s]*gu.quicksum(C_MIP[i,t]*w[i,t,s] for i in M for t in range(t_RH+1,t_max+1) ) for s in S)
            
        m.setObjective(compra_dia_t+ruta_dia_t+compa_other_days+ruta_other_days)
                
        m.update()
        m.setParam('OutputFlag',0)
        m.optimize()
        
        if m.Status==2 or m.Status==9:
            
            of_full_aproximation = m.ObjVal
            var_compra = {(i,k,t,s):z[i,k,t,s].x for t in TT for k in K for i in Mk[k,t] for s in S}
            
            total_compra_MIP = sum(ps[s]*sum(p[i,k,t_RH,path_eval]*z[i,k,t_RH,s].x for k in K for i in Mk[k,t_RH]) + sum(p_sample_paths[i,k,t,path_eval,s]*z[i,k,t,s].x for k in K for t in range(t_RH+1,t_max+1) for i in Mk[k,t] ) for s in S)
            total_ruta_MIP = sum(ps[s]*sum(C_MIP[i,t]*w[i,t,s].x for i in M for t in TT) for s in S)
            
            for k in K:
                compra_extra[t_RH][k] = bo[k,t_RH,0].x
                for o in range(O_k[k]):
                    inventario[t_RH][0][k][o] = I_0[k,o]
                    inventario[t_RH][1][k][o] = ii[k,t_RH,o,0].x
            
            for k in K:
                for i in Mk[k,t_RH]:
                    if var_compra[i,k,t_RH,0] > 0:
                        solucionTTP[t_RH][0][i] = True
                        solucionTTP[t_RH][1][i]+= var_compra[i,k,t_RH,0]
                        solucionTTP[t_RH][2][i][k]=True
                        solucionTTP[t_RH][3][i][k]=var_compra[i,k,t_RH,0]
                        solucionTTP[t_RH][6][k]+=var_compra[i,k,t_RH,0]
                        
            Rutas_finales, solucionTTP, solucionTTP[t_RH][8]  = Genera_ruta_at_t(solucionTTP, t_RH, max_cij, c, Q)
                    
            solucionTTP[t_RH].append(Rutas_finales.copy())
            
            costo_compra_dia_t = sum(p[i,k,t_RH,path_eval]*z[i,k,t_RH,0].x for k in K for i in Mk[k,t_RH])
            solucionTTP[t_RH][7] = costo_compra_dia_t
            costo_ruta_dia_t = solucionTTP[t_RH][8]
            
            final_policy[t_RH]=(solucionTTP[t_RH].copy(), inventario[t_RH].copy(), compra_extra[t_RH], costo_compra_dia_t, costo_ruta_dia_t, costo_compra_dia_t+costo_ruta_dia_t)
            FO_policy += costo_compra_dia_t+costo_ruta_dia_t
            
            for k in K:
                for o in range(O_k[k]):
                    I_0[k,o] = ii[k,t_RH,o,0].x
                          
        else:
            print('Problem infeasible')
            
    return final_policy, FO_policy

In [ ]:
Vertex = 5
Products = 4
Periods=5
paths_evaluation=2
veh_factor=3
seedd=1234
sample_paths=4 

Vertex, Products, Periods, M, V, K, T, coor, Mk, Q, O_k, h, d, d_EV, d_sample_paths, q, q_EV, q_sample_paths, p,p_EV, p_sample_paths, q_disp, I_0, c, max_cij, paths_evaluation, dist_demand_parm = Stochastic_Instance_Generator(Vertex, Products, Periods, paths_evaluation, veh_factor, seedd, sample_paths)
h = {(k,t): 0 for k in K for t in T} #For now holding cost is cero

In [ ]:
path_eval = 0 #Path evaluation
H = 2 #Size of windows
Final_policy_RHD, FO_policy_RHD = Deterministic_Rolling_Horizon(Vertex, Products, Periods, V, M, K, T, Q, h, O_k, Mk, d, q, p, q_disp, I_0, c, max_cij, dist_demand_parm, d_EV, q_EV, p_EV, H, path_eval)

In [ ]:
Final_policy_RHS, FO_policy_RHS = Stochasctic_Rolling_Horizon(Vertex, Products, Periods, Q, coor, h, d, O_k, Mk, q, p, M, V, K, T, q_disp, I_0, c, max_cij, dist_demand_parm, q_sample_paths, p_sample_paths, d_sample_paths, H, path_eval, sample_paths)